In [1]:
import json
import os
from glob import glob
import pandas as pd
import time
import xmltodict
import forecastio
import datetime
from pga.retrieve_course_info import parse_links

In [2]:
os.chdir('/home/valesco/Datasets/PGA_Data/PGA/')
pga_ls = glob('*')

os.chdir('/home/valesco/Datasets/PGA_Data/PGA_Dir/')
process_ls = glob('*')


In [13]:
os.chdir('/home/valesco/Datasets/PGA_Data/PGA_Dir/')
process_ls = ['002_2017']

succeed_ls = []
failed_ls = []
for i, tourny in enumerate(process_ls):
    #try:
    tourny_path = '/home/valesco/Datasets/PGA_Data/PGA/' + tourny
    os.chdir(tourny_path)

    # return tournament_code plus year
    tourny_prefix = tourny_path.replace('/home/valesco/Datasets/PGA_Data/PGA/','')


    # load course json file
    course_json = open(tourny_prefix + '_course.json')
    course_data = json.load(course_json)

    # load leaderboard json file
    leaderboard_json = open(tourny_prefix + '_leaderboard-v2.json')
    leaderboard_data = json.load(leaderboard_json)

    # load money json file
    money_json = open(tourny_prefix + '_money.json')
    money_data = json.load(money_json)

    # load player stats json file
    stats_json = open(tourny_prefix + '_player_stats.json')
    stats_data = json.load(stats_json)

    # load playoff json file
    playoff_json = open(tourny_prefix + '_playoff_info.json')
    playoff_data = json.load(playoff_json)

    # load teetimes json file
    tee_json = open(tourny_prefix + '_teetimes.json')
    tee_data = json.load(tee_json)

    # load tournsum json file
    tournsum_json = open(tourny_prefix + '_tournsum.json')
    tournsum_data = json.load(tournsum_json)

    # load course location file
    course_loc_json = open('/home/valesco/Datasets/PGA_Data/course_locations.json')
    course_loc_data = json.load(course_loc_json)

    # load player_id to player_name

    #Turn setup.xml to ordered dict to retrieve course grass type
    with open(tourny_prefix + '_setup.xml') as fd:
        setup_data = xmltodict.parse(fd.read())

    setup2_json = open(tourny_prefix + '_setup.json')
    setup2_data = json.load(setup2_json)

    # PlayerScoreCards Directory
    score_card_path = tourny_path + '/PlayerScoreCards/'

    tournament_dict = {}

    #Get tournament code, tourny, and course name
    tourny_code = tourny_prefix.split('_')[0]
    tourny_name = tee_data['tournament']['TournamentName']


    tournament_dict[tourny_code] = {'tournament_name': tourny_name}

    #Count number of courses played in tournament and dictate for json to use course or courses
    num_courses = len(course_data['courses'])

    if num_courses > 1:
        course_courses = 'courses'
    else:
        course_courses = 'course'

    #make dict of course info for each course in tournament
    tourny_course_dict = {}
    for e, course in enumerate(course_data['courses']):
        tourny_course_name = course_data['courses'][e]['name']
        tourny_course_num = course_data['courses'][e]['number']
        tourny_par_front = tee_data['tournament']['rounds'][0]['courses'][e]['Front']
        tourny_par_back = tee_data['tournament']['rounds'][0]['courses'][e]['Back']
        tourny_yardage = course_data['courses'][e]['yards']
        tourny_course_num = tournsum_data['years'][0]['tours'][0]['trns'][0]['courses'][e]['courseNum']
        if 'stimp' in leaderboard_data['leaderboard']['courses'][e]['holes'][0]['round'][0].keys():
            tourny_stimp = leaderboard_data['leaderboard']['courses'][e]['holes'][0]['round'][0]['stimp']
        else:
            tourny_stimp = 'no_stimp_reading'

        try:
            if 'grassTypes' in setup_data['trn']['courseInfo']['course'][e].keys():
                tourny_grass_fairway = setup_data['trn']['courseInfo']['course'][e]['grassTypes']['@fairway']
                tourny_grass_green = setup_data['trn']['courseInfo']['course'][e]['grassTypes']['@green']
                tourny_grass_rough = setup_data['trn']['courseInfo']['course'][e]['grassTypes']['@rough']
            else:
                tourny_grass_fairway = 'no_grass_type'
                tourny_grass_green = 'no_grass_type'
                tourny_grass_rough = 'no_grass_type'

        except:      
            if 'grassTypes' in setup_data['trn']['courseInfo']['course'].keys():
                tourny_grass_fairway = setup_data['trn']['courseInfo']['course']['grassTypes']['@fairway']
                tourny_grass_green = setup_data['trn']['courseInfo']['course']['grassTypes']['@green']
                tourny_grass_rough = setup_data['trn']['courseInfo']['course']['grassTypes']['@rough']
            else:
                tourny_grass_fairway = 'no_grass_type'
                tourny_grass_green = 'no_grass_type'
                tourny_grass_rough = 'no_grass_type'

        #tourny_course_loc = 'no_course_loc_info'
        tourny_course_loc = []
        for course in course_loc_data['courses']:
            if course['courseNum'] == tourny_course_num:
                tourny_course_loc = course


        temp_dict = {'course_name': tourny_course_name, 'par_front': \
            tourny_par_front, 'par_back': tourny_par_back, 'course_yards': tourny_yardage,
            'stimp': tourny_stimp, 'grass_type': {'fairway_grass': tourny_grass_fairway, 'green_grass': tourny_grass_green,
            'rough_grass': tourny_grass_rough}, 'course_loc': tourny_course_loc, 'course_num': tourny_course_num}

        # set default key as list, and append all courses to the course key
        tourny_course_dict['_num_courses'] = str(len(course_data['courses']))  
        tourny_course_dict.setdefault('course', []).append(temp_dict)

    tournament_dict[tourny_code]['course_info'] = tourny_course_dict


    # add field info
    field_dict = {}
    for s in setup2_data['trn']['field']:
        player_id = s['id']
        player_first_name = s['name']['first']
        player_last_name = s['name']['last']
        player_ytd_money_rank = s['money']['ytdRank']
        player_current_country = s['info']['country']
        player_country_birth = s['info']['countryOfBirth']
        player_hometown = s['info']['hometown']
        player_status = s['status']
        field_dict[player_id] = {'first_name': player_first_name, 'last_name': player_last_name,
               'ytd_money_rank': player_ytd_money_rank, 'current_country': player_current_country, 
               'birth_country': player_country_birth, 'hometown': player_hometown, 'playing_status': player_status}

    tournament_dict[tourny_code]['tee_data'] = tee_data['tournament']
    tournament_dict[tourny_code]['field'] = field_dict


    tourny_start_date = tee_data['tournament']['rounds'][0]['courses'][0]['segments'][0]['groups'][0]['StartDate']
    tournament_dict[tourny_code]['start_date'] = tourny_start_date

    # Add weather dictionary
    api = 'c7c2d4dea244baeca35b91ed97364062'
    weather_dict = {}
    for d in range(4):
        lat = tourny_course_loc['latitude']
        lon = tourny_course_loc['longitude']
        date_object = datetime.datetime.strptime(tourny_start_date, '%m/%d/%Y')
        date_string = date_object + datetime.timedelta(days = d)
        weather = forecastio.load_forecast(api, lat, lon, time = date_string, units = 'us')
        hourly = weather.hourly()

        temp_weather_ls = []
        for hour in hourly.data:
            h = hour.d
            h['time_str'] = datetime.datetime.fromtimestamp(int(h['time'])).strftime('%m/%d/%Y %H:%M')
            temp_weather_ls.append(h)

        weather_dict[d] = temp_weather_ls

    tournament_dict[tourny_code]['weather'] = weather_dict

    base_path = '/home/valesco/Datasets/PGA_Data/PGA_Dir'
    os.chdir(base_path)

    make_path = base_path + '/' + tourny_prefix
    try:
        os.chdir(make_path)
    except:
        os.mkdir(make_path)
        os.chdir(make_path)

    file_name = tourny_prefix + '_course_data1.json'
    with open(file_name, 'w') as outfile:
        json.dump(tournament_dict, outfile)

    print(tourny_prefix, ' succeeded!!')
    succeed_ls.append(tourny)
    #except Exception as e:
        #print(tourny_prefix, ' failed ', e)
        #failed_ls.append(tourny)

002_2017  succeeded!!


In [16]:
def parse_course_info(tourny):
    tourny_path = '/home/valesco/Datasets/PGA_Data/PGA/' + tourny
    os.chdir(tourny_path)

    # return tournament_code plus year
    tourny_prefix = tourny_path.replace('/home/valesco/Datasets/PGA_Data/PGA/','')


    # load course json file
    course_json = open(tourny_prefix + '_course.json')
    course_data = json.load(course_json)

    # load leaderboard json file
    leaderboard_json = open(tourny_prefix + '_leaderboard-v2.json')
    leaderboard_data = json.load(leaderboard_json)

    # load money json file
    money_json = open(tourny_prefix + '_money.json')
    money_data = json.load(money_json)

    # load player stats json file
    stats_json = open(tourny_prefix + '_player_stats.json')
    stats_data = json.load(stats_json)

    # load playoff json file
    playoff_json = open(tourny_prefix + '_playoff_info.json')
    playoff_data = json.load(playoff_json)

    # load teetimes json file
    tee_json = open(tourny_prefix + '_teetimes.json')
    tee_data = json.load(tee_json)

    # load tournsum json file
    tournsum_json = open(tourny_prefix + '_tournsum.json')
    tournsum_data = json.load(tournsum_json)

    # load course location file
    course_loc_json = open('/home/valesco/Datasets/PGA_Data/course_locations.json')
    course_loc_data = json.load(course_loc_json)
    
    # Check to see if course is in course_loc_data, if not, call parse_links to retrieve the data
    
    num_courses = len(tournsum_data['years'][0]['tours'][0]['trns'][0]['courses'])
    tourny_name = tournsum_data['years'][0]['tours'][0]['trns'][0]['fullName']


    course_info_ls = []
    valid_coords = []
    invalid_coords = []

    for c, course in enumerate(tournsum_data['years'][0]['tours'][0]['trns'][0]['courses']):
        course_name = course['courseName']
        course_num= course['courseNum']

        course_loc_count = 0
        for course_loc in course_loc_data['courses']:
            if course_loc['courseNum'] == course_num:
                course_loc_count += 1

        if course_loc_count == 0:
            temp_course_info = parse_links(tourny_name, course_name, course_num)

            if temp_course_info is not None:
                valid_coords.append(c)
                course_info_ls.append(temp_course_info)
            else:
                invalid_coords.append(c)
                course_info_ls.append({'name': course_name, 'city': 'nan', 'country': 'nan', 'courseNum': course_num,
                                      'latitude': 'nan', 'longitude': 'nan', 'state': 'nan', 'timezone': 'nan'})


    for invalid in invalid_coords:
        if len(valid_coords) > 0:
            course_info_ls[invalid]['city'] = course_info_ls[valid_coords[0]]['city']
            course_info_ls[invalid]['country'] = course_info_ls[valid_coords[0]]['country']
            course_info_ls[invalid]['state'] = course_info_ls[valid_coords[0]]['state']
            course_info_ls[invalid]['latitude'] = course_info_ls[valid_coords[0]]['latitude']
            course_info_ls[invalid]['longitude'] = course_info_ls[valid_coords[0]]['longitude']
            course_info_ls[invalid]['timezone'] = course_info_ls[valid_coords[0]]['timezone']

    if len(course_info_ls) > 0:
        course_loc_data.extend(course_info_ls)
        with open('/home/valesco/Datasets/PGA_Data/course_locations.json', 'w') as outfile:
            json.dump(course_loc_data, outfile)

    # load player_id to player_name

    #Turn setup.xml to ordered dict to retrieve course grass type
    with open(tourny_prefix + '_setup.xml') as fd:
        setup_data = xmltodict.parse(fd.read())

    setup2_json = open(tourny_prefix + '_setup.json')
    setup2_data = json.load(setup2_json)

    # PlayerScoreCards Directory
    score_card_path = tourny_path + '/PlayerScoreCards/'

    tournament_dict = {}

    #Get tournament code, tourny, and course name
    tourny_code = tourny_prefix.split('_')[0]
    tourny_name = tee_data['tournament']['TournamentName']


    tournament_dict[tourny_code] = {'tournament_name': tourny_name}

    #Count number of courses played in tournament and dictate for json to use course or courses
    num_courses = len(course_data['courses'])

    if num_courses > 1:
        course_courses = 'courses'
    else:
        course_courses = 'course'

    #make dict of course info for each course in tournament
    tourny_course_dict = {}
    for e, course in enumerate(course_data['courses']):
        tourny_course_name = course_data['courses'][e]['name']
        tourny_course_num = course_data['courses'][e]['number']
        tourny_par_front = tee_data['tournament']['rounds'][0]['courses'][e]['Front']
        tourny_par_back = tee_data['tournament']['rounds'][0]['courses'][e]['Back']
        tourny_yardage = course_data['courses'][e]['yards']
        tourny_course_num = tournsum_data['years'][0]['tours'][0]['trns'][0]['courses'][e]['courseNum']
        if 'stimp' in leaderboard_data['leaderboard']['courses'][e]['holes'][0]['round'][0].keys():
            tourny_stimp = leaderboard_data['leaderboard']['courses'][e]['holes'][0]['round'][0]['stimp']
        else:
            tourny_stimp = 'no_stimp_reading'

        try:
            if 'grassTypes' in setup_data['trn']['courseInfo']['course'][e].keys():
                tourny_grass_fairway = setup_data['trn']['courseInfo']['course'][e]['grassTypes']['@fairway']
                tourny_grass_green = setup_data['trn']['courseInfo']['course'][e]['grassTypes']['@green']
                tourny_grass_rough = setup_data['trn']['courseInfo']['course'][e]['grassTypes']['@rough']
            else:
                tourny_grass_fairway = 'no_grass_type'
                tourny_grass_green = 'no_grass_type'
                tourny_grass_rough = 'no_grass_type'

        except:      
            if 'grassTypes' in setup_data['trn']['courseInfo']['course'].keys():
                tourny_grass_fairway = setup_data['trn']['courseInfo']['course']['grassTypes']['@fairway']
                tourny_grass_green = setup_data['trn']['courseInfo']['course']['grassTypes']['@green']
                tourny_grass_rough = setup_data['trn']['courseInfo']['course']['grassTypes']['@rough']
            else:
                tourny_grass_fairway = 'no_grass_type'
                tourny_grass_green = 'no_grass_type'
                tourny_grass_rough = 'no_grass_type'

        tourny_course_loc = 0
        for course in course_loc_data['courses']:
            if course['courseNum'] == tourny_course_num:
                tourny_course_loc = course

        if tourny_course_loc == 0:
            print('No course found {}'.format(course['courseNum']))
        temp_dict = {'course_name': tourny_course_name, 'par_front': \
            tourny_par_front, 'par_back': tourny_par_back, 'course_yards': tourny_yardage,
            'stimp': tourny_stimp, 'grass_type': {'fairway_grass': tourny_grass_fairway, 'green_grass': tourny_grass_green,
            'rough_grass': tourny_grass_rough}, 'course_loc': tourny_course_loc, 'course_num': tourny_course_num}

        # set default key as list, and append all courses to the course key
        tourny_course_dict['_num_courses'] = str(len(course_data['courses']))  
        tourny_course_dict.setdefault('course', []).append(temp_dict)

    tournament_dict[tourny_code]['course_info'] = tourny_course_dict


    # add field info
    field_dict = {}
    for s in setup2_data['trn']['field']:
        player_id = s['id']
        player_first_name = s['name']['first']
        player_last_name = s['name']['last']
        player_ytd_money_rank = s['money']['ytdRank']
        player_current_country = s['info']['country']
        player_country_birth = s['info']['countryOfBirth']
        player_hometown = s['info']['hometown']
        player_status = s['status']
        field_dict[player_id] = {'first_name': player_first_name, 'last_name': player_last_name,
               'ytd_money_rank': player_ytd_money_rank, 'current_country': player_current_country, 
               'birth_country': player_country_birth, 'hometown': player_hometown, 'playing_status': player_status}

    tournament_dict[tourny_code]['tee_data'] = tee_data['tournament']
    tournament_dict[tourny_code]['field'] = field_dict


    tourny_start_date = tee_data['tournament']['rounds'][0]['courses'][0]['segments'][0]['groups'][0]['StartDate']
    tournament_dict[tourny_code]['start_date'] = tourny_start_date

    # Add weather dictionary
    api = 'c7c2d4dea244baeca35b91ed97364062'
    weather_dict = {}
    for d in range(4):
        lat = tourny_course_loc['latitude']
        lon = tourny_course_loc['longitude']
        date_object = datetime.datetime.strptime(tourny_start_date, '%m/%d/%Y')
        date_string = date_object + datetime.timedelta(days = d)
        weather = forecastio.load_forecast(api, lat, lon, time = date_string, units = 'us')
        hourly = weather.hourly()

        temp_weather_ls = []
        for hour in hourly.data:
            h = hour.d
            h['time_str'] = datetime.datetime.fromtimestamp(int(h['time'])).strftime('%m/%d/%Y %H:%M')
            temp_weather_ls.append(h)

        weather_dict[d] = temp_weather_ls

    tournament_dict[tourny_code]['weather'] = weather_dict

    base_path = '/home/valesco/Datasets/PGA_Data/PGA_Dir'
    os.chdir(base_path)

    make_path = base_path + '/' + tourny_prefix
    try:
        os.chdir(make_path)
    except:
        os.mkdir(make_path)
        os.chdir(make_path)

    file_name = tourny_prefix + '_course_data1.json'
    with open(file_name, 'w') as outfile:
        json.dump(tournament_dict, outfile)

    print(tourny_prefix, ' succeeded!!')


In [17]:
#succeed_ls = []
#failed_ls = []
for tourny in failed_ls:
    try:
        parse_course_info(tourny)
        succeed_ls.append(tourny)
    except Exception as e:
        print(tourny, ' failed {}'.format(e))
        #failed_ls.append(tourny)

473_2017  succeeded!!
480_2017  succeeded!!
464_2015  succeeded!!
464_2016  succeeded!!


In [7]:
parse_links('The Genesis Open','Riviera CC','500')

{'city': 'Los Angeles',
 'country': 'United States',
 'courseNum': '500',
 'latitude': 34.05,
 'longitude': -118.5,
 'name': 'Riviera Country Club',
 'state': 'California',
 'timezone': 'America/Los_Angeles'}

In [14]:
for tourny in process_ls:
    course_file = open('/home/valesco/Datasets/PGA_Data/PGA_Dir/{}/{}_course_data1.json'.format(tourny))
    course_data = json.load(course_file)
    
    tourny_num = tourny.split('_')[0]
    tourny_year = tourny.split('_')[1]
    
    tournament_name = course_data[tourny_num]['tournament_name']
    num_courses = int(course_data[tourny_num]['course_info']['num_courses'])
    
    course_num_ls = []
    course_name_ls = []
    for c, course in course_data[tourny_num]['course_info']['course']:
        
        

In [19]:
tournament_name = course_data['002']['tournament_name']
course_data['002']['course_info']['course']

[{'course_loc': {'city': 'La Quinta',
   'country': 'USA',
   'courseNum': '704',
   'latitude': '33.643302',
   'longitude': '-116.267506',
   'name': 'PGA West Stadium Course',
   'state': 'CA',
   'timezone': 'America/Los_Angeles'},
  'course_name': 'Stadium Course',
  'course_num': '704',
  'course_yards': '7,300',
  'grass_type': {'fairway_grass': '328 Bermudagrass',
   'green_grass': 'Tifdwarf Bermuda/Ryegrass/Poa Trivilias',
   'rough_grass': '328 Bermudagrass'},
  'par_back': '36',
  'par_front': '36',
  'stimp': '11'},
 {'course_loc': {'city': 'La Quinta',
   'country': 'USA',
   'courseNum': '233',
   'latitude': '33.643302',
   'longitude': '-116.267506',
   'name': 'PGA West Nicklaus Course',
   'state': 'CA',
   'timezone': 'America/Los_Angeles'},
  'course_name': 'Nicklaus Tournament Course',
  'course_num': '233',
  'course_yards': '7204',
  'grass_type': {'fairway_grass': '328 Bermudagrass',
   'green_grass': 'Tifdwarf Bermuda/Ryegrass/Poa Trivilias',
   'rough_grass': 

In [31]:
course_json = open('/home/valesco/Datasets/PGA_Data/PGA/{}/{}'.format(tourny_prefix, tourny_prefix) + '_course.json')
course_data = json.load(course_json)

tournsum_json = open('/home/valesco/Datasets/PGA_Data/PGA/{}/{}'.format(tourny_prefix, tourny_prefix) + '_tournsum.json')
tournsum_data = json.load(tournsum_json)

In [52]:
num_courses = len(tournsum_data['years'][0]['tours'][0]['trns'][0]['courses'])
tourny_name = tournsum_data['years'][0]['tours'][0]['trns'][0]['fullName']


course_info_ls = []
valid_coords = []
invalid_coords = []

for c, course in enumerate(tournsum_data['years'][0]['tours'][0]['trns'][0]['courses']):
    course_name = course['courseName']
    course_num= course['courseNum']
    
    course_loc_count = 0
    for course_loc in course_loc_data['courses']:
        if course_loc['courseNum'] == course_num:
            course_loc_count += 1
    
    if course_loc_count == 0
        temp_course_info = parse_links(tourny_name, course_name, course_num)

        if temp_course_info is not None:
            valid_coords.append(c)
            course_info_ls.append(temp_course_info)
        else:
            invalid_coords.append(c)
            course_info_ls.append({'name': course_name, 'city': 'nan', 'country': 'nan', 'courseNum': course_num,
                                  'latitude': 'nan', 'longitude': 'nan', 'state': 'nan', 'timezone': 'nan'})
        
    
for invalid in invalid_coords:
    if len(valid_coords) > 0:
        course_info_ls[invalid]['city'] = course_info_ls[valid_coords[0]]['city']
        course_info_ls[invalid]['country'] = course_info_ls[valid_coords[0]]['country']
        course_info_ls[invalid]['state'] = course_info_ls[valid_coords[0]]['state']
        course_info_ls[invalid]['latitude'] = course_info_ls[valid_coords[0]]['latitude']
        course_info_ls[invalid]['longitude'] = course_info_ls[valid_coords[0]]['longitude']
        course_info_ls[invalid]['timezone'] = course_info_ls[valid_coords[0]]['timezone']
        
if len(course_info_ls) > 0:
    course_loc_data.extend(course_info_ls)
    with open('/home/valesco/Datasets/PGA_Data/course_locations.json', 'w') as outfile:
        json.dump(course_loc_data, outfile)

CareerBuilder Challenge or Nicklaus Tournament Course not found
CareerBuilder Challenge or La Quinta CC not found


[{'city': 'La Quinta',
  'country': 'United States',
  'courseNum': '704',
  'latitude': 33.643302,
  'longitude': -116.267506,
  'name': 'PGA West Stadium Course',
  'state': 'California',
  'timezone': 'America/Los_Angeles'},
 {'city': 'La Quinta',
  'country': 'United States',
  'courseNum': '233',
  'latitude': 33.643302,
  'longitude': -116.267506,
  'name': 'Nicklaus Tournament Course',
  'state': 'California',
  'timezone': 'America/Los_Angeles'},
 {'city': 'La Quinta',
  'country': 'United States',
  'courseNum': '202',
  'latitude': 33.643302,
  'longitude': -116.267506,
  'name': 'La Quinta CC',
  'state': 'California',
  'timezone': 'America/Los_Angeles'}]

In [53]:
course_loc_count = 0
for course_loc in course_loc_data['courses']:
    if course_loc['courseNum'] == course_num:
        course_loc_count += 1

{'courses': [{'city': '',
   'country': '',
   'courseNum': '775',
   'latitude': '37.067680',
   'longitude': '-121.632923',
   'name': '',
   'state': '',
   'timezone': 'America/Los_Angeles'},
  {'city': 'Napa',
   'country': 'USA',
   'courseNum': '552',
   'latitude': '38.348687',
   'longitude': '-122.264309',
   'name': 'Silverado CC (North)',
   'state': 'CA',
   'timezone': 'America/Los_Angeles'},
  {'city': '',
   'country': '',
   'courseNum': '765',
   'latitude': '32.260858',
   'longitude': '-64.874149',
   'name': '',
   'state': '',
   'timezone': 'Atlantic/Bermuda'},
  {'city': 'Las Vegas',
   'country': 'USA',
   'courseNum': '538',
   'latitude': '36.188528',
   'longitude': '-115.298280',
   'name': 'TPC Summerlin',
   'state': 'NV',
   'timezone': 'America/Los_Angeles'},
  {'city': 'Kuala Lumpur',
   'country': 'MAS',
   'courseNum': '800',
   'latitude': '3.138797',
   'longitude': '101.640407',
   'name': 'Kuala Lumpur G&CC',
   'state': '',
   'timezone': 'Asia/

In [10]:
course_data1 = open('/home/valesco/Datasets/PGA_Data/PGA_Dir/011_2017/011_2017_course_data1.json')
course_data1_json = json.load(course_data1)

In [11]:
course_data1_json

{'011': {'course_info': {},
  'field': {'01666': {'birth_country': 'GER',
    'current_country': 'GER',
    'first_name': 'Bernhard',
    'hometown': 'Munich, Germany',
    'last_name': 'Langer',
    'playing_status': {'alternate': 'n', 'amateur': 'n', 'member': 'y'},
    'ytd_money_rank': '242'},
   '01810': {'birth_country': 'USA',
    'current_country': 'USA',
    'first_name': 'Phil',
    'hometown': 'Rancho Santa Fe, CA',
    'last_name': 'Mickelson',
    'playing_status': {'alternate': 'n', 'amateur': 'n', 'member': 'y'},
    'ytd_money_rank': '40'},
   '06522': {'birth_country': 'RSA',
    'current_country': 'UNX',
    'first_name': 'Ernie',
    'hometown': 'South Africa',
    'last_name': 'Els',
    'playing_status': {'alternate': 'n', 'amateur': 'n', 'member': 'y'},
    'ytd_money_rank': '209'},
   '06527': {'birth_country': 'USA',
    'current_country': 'USA',
    'first_name': 'Steve',
    'hometown': 'Madison, WI',
    'last_name': 'Stricker',
    'playing_status': {'altern